# Notebook 3: Cosine Similarity Analysis and Visualization

**Objective:** Analyze occupational gender bias using cosine similarity between occupation embeddings and gender anchor embeddings ('he', 'she', 'man', 'woman'). This involves:
1. Loading the pre-generated GPT-2 embeddings and the validated occupation dictionary.
2. Calculating pairwise cosine similarities between each occupation and the four gender anchors.
3. Calculating aggregate male/female similarity scores and the resulting 'Similarity Bias'.
4. Visualizing the pairwise similarities using a heatmap (similar to Figure 3).
5. Visualizing the 'Similarity Bias' distribution using a bar plot (similar to Figure 4).
6. Saving the calculated similarity scores.

## 1. Import Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from pathlib import Path
import os

## 2. Configuration

In [2]:
# --- Paths ---
# Get project root assuming the notebook is in 'notebooks' directory
current_dir = Path.cwd()
project_root = current_dir.parent

In [3]:
# Input files
DICT_INPUT_CSV = project_root / 'results' / 'occupation_dictionary_validated.csv'
EMBEDDING_INPUT_FILE = project_root / 'results' / 'gpt2_static_embeddings.npz'

In [4]:
# Output files
RESULTS_DIR = project_root / 'results'
SIMILARITY_SCORES_OUTPUT_CSV = RESULTS_DIR / 'cosine_similarity_results.csv'
HEATMAP_OUTPUT_PNG = RESULTS_DIR / 'similarity_heatmap_figure3.png'
SIMILARITY_BIAS_BARPLOT_OUTPUT_PNG = RESULTS_DIR / 'similarity_bias_barplot_figure4.png'

In [5]:
# --- Parameters ---
GENDER_TERMS = ['he', 'she', 'man', 'woman']
# Number of occupations for the heatmap (adjust as needed)
HEATMAP_NUM_OCCUPATIONS = 30
# Number of occupations for the bar plot (top N/2 positive, bottom N/2 negative)
BARPLOT_NUM_OCCUPATIONS = 30

In [6]:
# Define specific colors for BLS labels for consistency (adjust if needed)
BLS_LABEL_COLORS = {
    'male-stereotyped': '#95B3D7',  # Blueish
    'neutral': '#9DCDA9',          # Greenish
    'female-stereotyped': '#FFB598' # Orangish/Reddish
    # Add 'unknown' if present and needs specific color
    # 'unknown': 'grey'
}

In [7]:
# Create results directory if it doesn't exist
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

## 3. Load Data

In [8]:
# Load occupation dictionary
try:
    df_dictionary = pd.read_csv(DICT_INPUT_CSV)
    print(f"Loaded dictionary with {len(df_dictionary)} occupations.")
except FileNotFoundError:
    print(f"Error: Dictionary file not found at {DICT_INPUT_CSV}")
    print("Please ensure Notebook 1 ran successfully.")
    raise
except Exception as e:
    print(f"Error loading dictionary CSV: {e}")
    raise

Loaded dictionary with 100 occupations.


In [9]:
# Load embeddings
try:
    embeddings_data = np.load(EMBEDDING_INPUT_FILE, allow_pickle=True)
    # Convert NpzFile items to a standard dictionary
    embeddings = {key: embeddings_data[key] for key in embeddings_data.files}
    print(f"Loaded {len(embeddings)} embeddings.")
except FileNotFoundError:
    print(f"Error: Embeddings file not found at {EMBEDDING_INPUT_FILE}")
    print("Please ensure Notebook 2 ran successfully.")
    raise
except Exception as e:
    print(f"Error loading embeddings file: {e}")
    raise

Loaded 104 embeddings.


In [10]:
# Verify essential gender embeddings are loaded
if not all(term in embeddings for term in GENDER_TERMS):
    missing_genders = [term for term in GENDER_TERMS if term not in embeddings]
    raise ValueError(f"Essential gender term embeddings missing from file: {missing_genders}")

## 4. Calculate Similarities and Bias

In [11]:
results_list = []
processed_occupations = set() # To handle potential duplicates in dictionary if not handled before

In [12]:
for index, row in df_dictionary.iterrows():
    occupation = row['occupation']

    # Skip if already processed (in case duplicates exist)
    if occupation in processed_occupations:
        continue
    processed_occupations.add(occupation)

    if occupation in embeddings:
        occ_emb = embeddings[occupation]
        # Reshape for cosine_similarity function (expects 2D arrays)
        occ_emb_2d = occ_emb.reshape(1, -1)

        similarities = {}
        try:
            # Calculate similarity with each gender term
            for term in GENDER_TERMS:
                gender_emb = embeddings[term]
                gender_emb_2d = gender_emb.reshape(1, -1)
                sim_score = cosine_similarity(occ_emb_2d, gender_emb_2d)[0][0]
                similarities[f'sim_{term}'] = sim_score # e.g., sim_he, sim_she

            # Calculate aggregate scores and bias
            male_sim = (similarities['sim_he'] + similarities['sim_man']) / 2.0
            female_sim = (similarities['sim_she'] + similarities['sim_woman']) / 2.0
            sim_bias = male_sim - female_sim

            results_list.append({
                'occupation': occupation,
                **similarities, # Add sim_he, sim_she, etc.
                'male_similarity_agg': male_sim,
                'female_similarity_agg': female_sim,
                'similarity_bias': sim_bias
            })
        except Exception as e:
             print(f"Error calculating similarity for '{occupation}': {e}")
    else:
        print(f"Warning: Embedding not found for occupation '{occupation}'. Skipping.")

In [13]:
# Convert results to DataFrame
df_results = pd.DataFrame(results_list)

In [14]:
if df_results.empty:
    print("Error: No similarity results were generated. Check input data and embeddings.")
else:
     print(f"Calculated similarity scores for {len(df_results)} occupations.")

Calculated similarity scores for 100 occupations.


## 5. Merge Results with Dictionary

In [15]:
# Merge calculated scores back into the main dictionary dataframe
df_merged = pd.merge(df_dictionary, df_results, on='occupation', how='inner')

In [16]:
print(f"Final merged dataframe shape: {df_merged.shape}")
print("Merged DataFrame sample:")
print(df_merged[['occupation', 'bls_label', 'similarity_bias', 'sim_he', 'sim_she', 'sim_man', 'sim_woman']].head())

Final merged dataframe shape: (100, 13)
Merged DataFrame sample:
            occupation           bls_label  similarity_bias    sim_he  \
0      chief executive             neutral        -0.000055  0.978794   
1              manager             neutral         0.000889  0.996040   
2    marketing manager             neutral        -0.004225  0.964611   
3        sales manager             neutral        -0.000962  0.976865   
4  fundraising manager  female-stereotyped        -0.005224  0.958652   

    sim_she   sim_man  sim_woman  
0  0.978539  0.984331   0.984696  
1  0.996083  0.998030   0.996209  
2  0.967078  0.975647   0.981630  
3  0.977531  0.984261   0.985520  
4  0.961820  0.970997   0.978276  


## 6. Save Calculated Scores

In [17]:
print(f"\nSaving calculated similarity scores to {SIMILARITY_SCORES_OUTPUT_CSV}...")
try:
    df_merged.to_csv(SIMILARITY_SCORES_OUTPUT_CSV, index=False, encoding='utf-8')
    print("Scores saved successfully.")
except Exception as e:
    print(f"Error saving scores CSV: {e}")


Saving calculated similarity scores to /Users/jessie/Documents/Projects/master_thesis_llms_bias/results/cosine_similarity_results.csv...
Scores saved successfully.


## 7. Generate Heatmap (Figure 3)

In [18]:
if df_merged.empty or 'similarity_bias' not in df_merged.columns:
    print("Skipping heatmap generation: No data or 'similarity_bias' column missing.")
else:
    # Sort by absolute similarity bias to select most biased (positive or negative)
    df_merged['abs_similarity_bias'] = df_merged['similarity_bias'].abs()
    df_sorted_heatmap = df_merged.sort_values('abs_similarity_bias', ascending=False)

    # Select top N occupations
    num_to_select_heatmap = min(HEATMAP_NUM_OCCUPATIONS, len(df_sorted_heatmap))
    if num_to_select_heatmap == 0:
        print("Skipping heatmap: No occupations to display.")
    else:
        df_heatmap_data = df_sorted_heatmap.head(num_to_select_heatmap)

        # Prepare data: index = occupation, columns = similarities
        heatmap_pivot = df_heatmap_data.set_index('occupation')[['sim_he', 'sim_she', 'sim_man', 'sim_woman']]
        # Rename columns for better display
        heatmap_pivot.columns = ['he', 'she', 'man', 'woman']

        # --- Create Plot ---
        try:
            # Adjust figure height based on number of occupations
            fig_height_heatmap = max(8, num_to_select_heatmap * 0.35)
            plt.figure(figsize=(10, fig_height_heatmap))

            # Calculate min/max for consistent color scaling, handle potential NaNs
            valid_data_heatmap = heatmap_pivot.values[~np.isnan(heatmap_pivot.values)]
            if valid_data_heatmap.size == 0:
                 print("Warning: No valid data for heatmap color scaling. Using default range.")
                 vmin, vmax = 0.95, 1.0 # Fallback based on paper observation
            else:
                 # Set tight bounds around the actual data range
                 vmin = np.nanmin(valid_data_heatmap)
                 vmax = np.nanmax(valid_data_heatmap)
                 # Optional: Add slight padding if desired
                 # padding = (vmax - vmin) * 0.01
                 # vmin -= padding
                 # vmax += padding

            ax_heatmap = sns.heatmap(
                heatmap_pivot,
                annot=True,       # Show similarity scores
                fmt=".4f",        # Format scores to 4 decimal places
                cmap="coolwarm",  # Colormap (red=high, blue=low)
                vmin=vmin, vmax=vmax, # Use calculated bounds for color scale
                linewidths=0.5,
                linecolor='white',
                cbar_kws={'label': 'Cosine Similarity'} # Label for the color bar
            )
            plt.title(f'Cosine Similarity between Top {num_to_select_heatmap} Occupations and Gender Terms\n(Sorted by Absolute Similarity Bias)', fontsize=14)
            plt.xlabel('Gender Terms', fontsize=12)
            plt.ylabel('Occupation', fontsize=12)
            plt.yticks(rotation=0) # Keep occupation names horizontal
            plt.tight_layout()

            # Save the plot
            plt.savefig(HEATMAP_OUTPUT_PNG, dpi=300, bbox_inches='tight')
            print(f"Heatmap saved successfully to {HEATMAP_OUTPUT_PNG}")
            plt.close() # Close the plot figure to free memory

        except Exception as e:
            print(f"Error generating heatmap: {e}")
            plt.close() # Ensure plot is closed even if error occurs

Heatmap saved successfully to /Users/jessie/Documents/Projects/master_thesis_llms_bias/results/similarity_heatmap_figure3.png


## 8. Generate Similarity Bias Bar Plot (Figure 4)

In [19]:
if df_merged.empty or 'similarity_bias' not in df_merged.columns:
    print("Skipping similarity bias bar plot generation: No data or 'similarity_bias' column missing.")
else:
    # Sort by similarity bias score
    df_sorted_barplot = df_merged.sort_values('similarity_bias', ascending=False).dropna(subset=['similarity_bias'])

    # Select top N/2 most positive and bottom N/2 most negative biased occupations
    n_each_barplot = BARPLOT_NUM_OCCUPATIONS // 2
    if len(df_sorted_barplot) < BARPLOT_NUM_OCCUPATIONS:
        print(f"Warning: Fewer than {BARPLOT_NUM_OCCUPATIONS} occupations with valid scores ({len(df_sorted_barplot)}). Plotting all available.")
        df_selected_barplot = df_sorted_barplot
    elif len(df_sorted_barplot) >= n_each_barplot * 2:
        top_n_barplot = df_sorted_barplot.head(n_each_barplot)
        bottom_n_barplot = df_sorted_barplot.tail(n_each_barplot)
        # Combine and re-sort for consistent order in plot
        df_selected_barplot = pd.concat([top_n_barplot, bottom_n_barplot]).sort_values('similarity_bias', ascending=False)
    else: # Handle cases with < n_each*2 but >= n_each
         print(f"Warning: Only {len(df_sorted_barplot)} occupations available for bar plot. Plotting all.")
         df_selected_barplot = df_sorted_barplot

    if df_selected_barplot.empty:
        print("Skipping similarity bias bar plot: No occupations selected.")
    else:
        # --- Create Plot ---
        try:
            # Adjust figure height
            fig_height_barplot = max(8, len(df_selected_barplot) * 0.30)
            plt.figure(figsize=(12, fig_height_barplot))

            ax_barplot = sns.barplot(
                x='similarity_bias',
                y='occupation',
                data=df_selected_barplot,
                hue='bls_label',    # Color bars by BLS stereotype label
                palette=BLS_LABEL_COLORS, # Use predefined colors
                dodge=False,        # Don't dodge bars when using hue for coloring same bar
                orient='h'
            )

            plt.title('Similarity-Based Gender Bias Scores (Higher = More Male-Associated)', fontsize=16)
            plt.xlabel('Similarity Bias Score (Avg Male Sim - Avg Female Sim)', fontsize=12)
            plt.ylabel('Occupation', fontsize=12)
            plt.axvline(x=0, color='grey', linestyle='--', linewidth=1) # Add line at zero bias

            # Handle legend
            handles, labels = ax_barplot.get_legend_handles_labels()
            # Create legend with unique labels only (if duplicates arise from hue)
            unique_labels = []
            unique_handles = []
            for handle, label in zip(handles, labels):
                 if label not in unique_labels:
                      unique_labels.append(label)
                      unique_handles.append(handle)
            ax_barplot.legend(unique_handles, unique_labels, title='BLS Label', bbox_to_anchor=(1.02, 1), loc='upper left')

            plt.tight_layout(rect=[0, 0, 0.88, 1]) # Adjust layout to make space for legend

            # Save the plot
            plt.savefig(SIMILARITY_BIAS_BARPLOT_OUTPUT_PNG, dpi=300, bbox_inches='tight')
            print(f"Similarity bias bar plot saved successfully to {SIMILARITY_BIAS_BARPLOT_OUTPUT_PNG}")
            plt.close() # Close the plot figure

        except Exception as e:
            print(f"Error generating similarity bias bar plot: {e}")
            plt.close() # Ensure plot is closed

Similarity bias bar plot saved successfully to /Users/jessie/Documents/Projects/master_thesis_llms_bias/results/similarity_bias_barplot_figure4.png
